# Belief

$$
\begin{align}
bel(x_t) & = p(x_t  | z_{1:t}, u_{1:t}) \\
\overline{bel}(x_t) & = p(x_t|z_{1:t-1}, u_{1:t})
\end{align}
$$



# Bayes Filter

![Bayes Filter Psuedo code](../images/bayesFilter.PNG)